### Restart and Run All Cells

In [3]:
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2024
quarter = 3
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time )

2025-02-15 15:21:21


In [5]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',   
                'latest_amt':'{:,}','previous_amt':'{:,}','inc_amt':'{:,}',
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%','inc_pct':'{:.2f}%'
              }

In [7]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2024 AND quarter <= 3) 
OR (year = 2024-1 AND quarter >= 3+1)
ORDER BY year DESC, quarter DESC



In [9]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.shape

(222, 5)

In [11]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2024 AND quarter <= 3-1) 
OR (year = 2024-1 AND quarter >= 3) 
ORDER BY year DESC, quarter DESC


In [13]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.sample(5).style.format(format_dict)

,name,year,quarter,q_amt,Counter
171,SPCG,8094,10,"1,161,115",4
80,HANA,8094,10,"1,548,281",4
64,ECL,8094,10,"70,080",4
199,TOP,8094,10,"25,181,677",4
46,CKP,8094,10,"1,177,550",4


In [15]:
dfp.name.unique().shape

(224,)

In [17]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlt)
stocks.shape

(226, 15)

In [19]:
sql = """
SELECT *
FROM tickers
"""
tickers = pd.read_sql(sql, conlt)
tickers.shape

(394, 9)

In [21]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
#df_percent.style.format(format_dict)
df_percent.shape

(222, 7)

In [23]:
# Create the SQL query with parameter binding
sql = text("DELETE FROM qt_profits WHERE year = :year AND quarter = :quarter")

# Execute the query with parameters
params = {'year': year, 'quarter': f'Q{quarter}'}
rp = conlt.execute(sql, params)

# Print the number of rows affected
print("Rows deleted:", rp.rowcount)

Rows deleted: 212


In [25]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
tickers.sample(5)

,name,id
316,SVH,490
345,TNH,548
22,APCS,27
120,GFPT,187
373,UNIQ,599


In [27]:
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
#df_ins.style.format(format_dict)
df_ins.shape

(222, 8)

In [29]:
rcds = df_ins.values.tolist()
len(rcds)

222

In [31]:
# Convert DataFrame to list of records
rcds = df_ins.values.tolist()

# Define column names in the same order as values
columns = ['name', 'year', 'quarter', 'latest_amt', 'previous_amt', 'inc_amt', 'inc_pct', 'ticker_id']

# SQL insert statement with named parameters
sql = text("""
    INSERT INTO qt_profits 
    (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)
    VALUES (:name, :year, :quarter, :latest_amt, :previous_amt, :inc_amt, :inc_pct, :ticker_id)
""")

try:
    # Execute inserts
    for rcd in rcds:
        # Convert list to dictionary
        params = dict(zip(columns, rcd))
        conlt.execute(sql, params)
except Exception as e:
    raise e

### End of loop

In [34]:
criteria_1 = df_ins.q_amt_c > 440_000
#df_ins.loc[criteria_1, cols].style.format(format_dict)
df_ins.shape

(222, 8)

In [36]:
criteria_2 = df_ins.q_amt_p > 400_000
#df_ins.loc[criteria_2, cols].style.format(format_dict)
df_ins.shape

(222, 8)

In [38]:
criteria_3 = df_ins.percent > 10.00
#df_ins.loc[criteria_3, cols].style.format(format_dict)
df_ins.shape

(222, 8)

In [40]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
#df_ins.loc[df_ins_criteria, cols].style.format(format_dict)
df_ins.shape

(222, 8)

In [42]:
df_ins[df_ins_criteria].sort_values(by=["percent"], ascending=[False]).head(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
144,RCL,2024,Q3,"5,280,406","1,774,094","3,506,312",197.64%,396
50,CPF,2024,Q3,"15,506,673","6,386,787","9,119,886",142.79%,117
146,ROJNA,2024,Q3,"3,647,560","1,652,478","1,995,082",120.73%,404
181,TASCO,2024,Q3,"1,073,061","540,207","532,854",98.64%,502
45,CK,2024,Q3,"1,773,101","1,406,435","366,666",26.07%,106


In [44]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
9,AMATA,2024,Q3,"2,142,951","1,774,219","368,732",20.78%,21
13,ASIAN,2024,Q3,"815,672","736,167","79,505",10.80%,36
16,ASW,2024,Q3,"1,783,985","1,501,832","282,153",18.79%,728
45,CK,2024,Q3,"1,773,101","1,406,435","366,666",26.07%,106
46,CKP,2024,Q3,"1,346,529","1,177,550","168,979",14.35%,107


In [46]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
9,AMATA,2024,Q3,"2,142,951","1,774,219","368,732",20.78%,21
13,ASIAN,2024,Q3,"815,672","736,167","79,505",10.80%,36
16,ASW,2024,Q3,"1,783,985","1,501,832","282,153",18.79%,728
45,CK,2024,Q3,"1,773,101","1,406,435","366,666",26.07%,106
46,CKP,2024,Q3,"1,346,529","1,177,550","168,979",14.35%,107


In [48]:
sql = """
SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 2
ORDER BY COUNT(*) DESC, name
"""
print(sql)

tmp = pd.read_sql(sql, conlt)
tmp


SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 2
ORDER BY COUNT(*) DESC, name



,name,Nbr. of Group of 4 Quarters
0,BAT-3K,2


In [50]:
conlt.commit()
conlt.close()